In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_config = AmlCompute.provisioning_configuration(vm_size= "Standard_D2_V2")
cpu_cluster = ComputeTarget.create(ws,project_compute,compute_config)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling()

# Specify a Policy
policy = BanditPolicy()

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(compute_target = cpu_cluster,entry_script= 'train')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, hyperparameter_sampling= ps, policy=policy)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azurelml.widgets import RunDetails
HyperDriveRun(exp,hyperdrive_config= hyperdrive_config)
RunDetails(hyperdrive_config).show()

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = get_best_run_by_primary_metric()
joblib.dump(saved_model)

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv",separator=',')

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(data)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='mean_squared_error',
    training_data=x,
    label_column_name=y,
    n_cross_validations=5)

In [2]:
# Submit your automl run
from azureml.train.automl.run import AutoMLRun
### YOUR CODE HERE ###
# ws = Workspace.from_config()
# experiment = ws.experiments['exp']
automl_run = AutoMLRun(experiment)

In [ ]:
# Retrieve and save your best automl model.
best = automl_run.get_best_child()
### YOUR CODE HERE ###
joblib.dump(automl_saved_model)